In [1]:
import datetime

import numpy as np
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler, OrdinalEncoder
from xgboost import XGBRegressor


In [2]:
import shutil

In [3]:
shutil.unpack_archive(
    "/content/drive/MyDrive/Data Car Resale Value Prediction (3).zip", 
    extract_dir="."
    )

In [39]:
data = pd.read_csv(
    "Data/autos.csv", encoding="Latin"
)

In [40]:
def infer_columns(X: pd.DataFrame):
    cat_cols = list()
    for colname in X.columns:
        subset = X.loc[:, colname]
        if subset.dtype == "object" or subset.dtype == "category":
            uniq_len = len(X.loc[:, colname].unique())
            print(colname, uniq_len)
            if uniq_len < 10:
                cat_cols.append(colname)

    for catcol in cat_cols:
        X.loc[:, catcol] = X.loc[:, catcol].astype("category")


infer_columns(data)


def convert_appropriate_dtypes(X:pd.DataFrame):
    X["dateCrawled"] = pd.to_datetime(X.dateCrawled)
    X["lastSeen"] = pd.to_datetime(X.lastSeen)
    X["monthOfRegistration"] = pd.to_numeric(X.monthOfRegistration, downcast="integer")
    X["dateCreated"] = pd.to_datetime(X.dateCreated)


convert_appropriate_dtypes(data)


def remove_redundant_data(X: pd.DataFrame):
    # Seller contains only 1 type of data and so remove it.
    X.drop("seller", axis=1, inplace=True)

    # Offer contains Most of the data as Angebot and 12 data as Gesuch so remove it
    X.drop("offerType", axis=1, inplace=True)

    # Both DateCreated and DateCrawled are correlated
    X.drop("dateCrawled", axis=1, inplace=True)

    # nrOfPicture contains no information since it has only 1 data i.e) 0
    X.drop("nrOfPictures", axis=1, inplace=True)


remove_redundant_data(data)

def filter_registration_year(X: pd.DataFrame):
    # REDUCE DATA RANGE BY YEAR OF REGISTRATION
    mini = 1900
    current_year = datetime.date.today().year

    X = X.loc[(X.yearOfRegistration > mini) & (X.yearOfRegistration < current_year)]

    return X

data = filter_registration_year(data)


def translate(X: pd.DataFrame):
    nrd_translate = {
        "ja": "Yes",
        "nein": "no",
    }

    X.notRepairedDamage.replace(nrd_translate, inplace=True)

    gb_translate = {
        "automatik": "automatic",
        "manuell": "manual"
    }

    X.gearbox.replace(gb_translate, inplace=True)

    fueltype_translate = {
        "benzin": "petrol",
        "andere": "other",
        "elektro": "electro"
    }

    X.fuelType.replace(fueltype_translate, inplace=True)

    
translate(data)

dateCrawled 280500
name 233531
seller 2
offerType 2
abtest 2
vehicleType 9
gearbox 3
model 252
fuelType 8
brand 40
notRepairedDamage 3
dateCreated 114
lastSeen 182806


/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:6619: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


In [50]:
data["gearbox"] = data.gearbox.astype("category")
data["fuelType"] = data.gearbox.astype("category")
data["notRepairedDamage"] = data.notRepairedDamage.astype("category")

array([nan, 'Yes', 'no'], dtype=object)

In [51]:
target = "price"
X = data.drop(target, axis=1)
y = data.loc[:, target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [53]:



feats_to_use = X_train.columns.drop(["name", "postalCode"])


num_cols = X_train.select_dtypes(include = [int, float]).columns
cat_cols = X_train.select_dtypes(include=["category"]).columns
object_cols = X_train.loc[:,["model", "brand"]].columns


numeric_pipeline = Pipeline(steps=[
    ('impute', SimpleImputer(strategy='mean')),
    ('scale', StandardScaler())
])


categorical_pipeline = Pipeline(steps=[
    ('impute', SimpleImputer(strategy='most_frequent')),
    ('one-hot', OneHotEncoder())
])

object_pipeline = Pipeline(steps=[
    ('impute', SimpleImputer(strategy='most_frequent')),
    ('one-hot', OrdinalEncoder())
])

In [54]:

full_processor = ColumnTransformer(transformers=[
    ('number', numeric_pipeline, num_cols),
    ('category', categorical_pipeline, cat_cols),
    ("object", object_pipeline, object_cols)
])


model_pipeline = Pipeline([
    ("prep_pipe", full_processor),
    ("model", RandomForestRegressor(n_jobs=-1, max_depth=30,max_features="log2"))
])

model_pipeline.fit(X_train, y_train)

Pipeline(steps=[('prep_pipe',
                 ColumnTransformer(transformers=[('number',
                                                  Pipeline(steps=[('impute',
                                                                   SimpleImputer()),
                                                                  ('scale',
                                                                   StandardScaler())]),
                                                  Index(['yearOfRegistration', 'powerPS', 'kilometer', 'postalCode'], dtype='object')),
                                                 ('category',
                                                  Pipeline(steps=[('impute',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('one-hot',
                                                                   OneHotEncoder())]),
                                   

In [60]:
import plotly.express as px

In [71]:
data.loc[data.price > 7000000]

,name,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,postalCode,lastSeen
14663,Tausch_moeglich_Polo_vw_n9,11111111,control,coupe,2003,manual,64,polo,150000,2,manual,volkswagen,NaN,2016-04-03,46535,2016-04-07 10:16:51
20746,Golf3_Schlachtfest,12345678,test,NaN,1995,NaN,0,NaN,150000,0,NaN,volkswagen,NaN,2016-03-08,18516,2016-03-17 16:15:43
21467,Porsche_911_911R_1_of_20_ORIGINAL_R_nur_798KG!...,9999999,control,coupe,1967,manual,215,911,50000,10,manual,porsche,no,2016-03-19,70435,2016-04-01 10:45:30
39625,Tausche_volvo_v40_gegen_van,12345678,control,NaN,2018,manual,95,v40,150000,6,manual,volvo,no,2016-03-08,14542,2016-04-06 23:17:31
54295,Mazda_5_2.0_CD_DPF_Exclusive,10000000,control,bus,2007,manual,143,5_reihe,150000,2,manual,mazda,no,2016-03-08,83301,2016-03-10 14:17:16
54507,Tausch_DKW_F8_gegen_Lanz_Bulldog,11111111,test,andere,1953,manual,20,NaN,150000,5,manual,audi,no,2016-03-09,7774,2016-04-05 14:46:32
56973,tausche_ford_mondeo_mk3_ghia_gegen_anderes_auto,99999999,control,limousine,2002,manual,131,mondeo,150000,1,manual,ford,NaN,2016-03-18,27574,2016-03-18 18:52:27
69747,Suche_Mercedes_Benz_r_klasse_cdi,99999999,control,bus,2006,NaN,0,andere,150000,0,NaN,mercedes_benz,NaN,2016-03-20,86916,2016-04-06 07:45:57
77520,Tausch_gegen_gleichwertiges,99999999,control,limousine,1999,automatic,224,s_klasse,150000,9,automatic,mercedes_benz,NaN,2016-03-22,73525,2016-04-06 05:15:30
87799,Leasinguebernahme,27322222,control,limousine,2014,manual,163,c4,40000,2,manual,citroen,NaN,2016-03-08,76532,2016-03-08 20:39:05


In [62]:
data.price.describe()

count    3.713460e+05
mean     1.729561e+04
std      3.588832e+06
min      0.000000e+00
25%      1.150000e+03
50%      2.950000e+03
75%      7.200000e+03
max      2.147484e+09
Name: price, dtype: float64

In [74]:
pred = model_pipeline.predict(X_test)


1355955368067.4333


In [76]:
print( mean_absolute_error(y_test, pred) )
print( median_absolute_error(y_test, pred) )

20619.88135522712
640.6999144707661


In [72]:
from sklearn.metrics import median_absolute_error, mean_absolute_error

pred = model_pipeline.predict(X_train)



7609.7298841252405
313.0482041740918
0.852862255017661
